# Generating puzzles with multiple solutions
Puzzles with multiple solutions can create very interesting effects, such as the puzzle shown in the video "[How can a jigsaw have two distinct solutions?](https://www.youtube.com/watch?v=b5nElEbbnfU)" on the Stand-Up Maths Youtube channel where the two ways of solving the jigsaw puzzle either create an image of a coffee cup or a donut. This page describes how such puzzles can be generated by first picking a random scrambling of the puzzle pieces, and then generating a puzzle that is solvable in both its unscrambled and scrambled state. There is also Python code.

# Describing puzzles using vectors
In order for a jigsaw puzzle to be solvable in more than one way, there must be several connections between pieces that have the same shape. Otherwise, if every connection was unique, each side of a piece would only be able to connect with a single side of a single other piece, and there would only be a single solution to the puzzle. This means that each side of each puzzle piece can be grouped with other puzzle piece sides with a similar shape, and these groups can be given numbers.

It is useful to give the shapes numbers so that shapes that are compatible with each other have inverse numbers. For example, shape 1 is compatible with shape -1, shape 2 is compatible with shape -2, and so forth. This makes it easy to check whether two shapes are compatible: they are compatible if they add up to 0, and incompatible if they do not. I prefer to think of the positive numbers as all the shapes that stick out and the negative numbers as the indented shapes, but this is fully arbitrary. Finally, the outer sides of the pieces at the edge of the jigsaw puzzle are just shaped like straight lines, and do not connect to anything. These non-connecting shapes are labelled 0.

A puzzle can be described mathematically as a vector containing the shapes of each side of all the pieces of the puzzle. The order of the sides is also arbitrary, but here we order the sides in order left, top, right, bottom, starting with the top left piece and going row by row. The ordering of sides for a 2x2 puzzle is illustrated in the following figure.

<img src="img/side_numbers.png" style="width:500px; margin-left:auto; margin-right:auto;"/>

Thus, if we assume that all the tabs in the image are in shape group 1 and all the indentations are in shape group -1, the vector describing this puzzle is:
$$
    \vec{p} = \begin{pmatrix}
        0 \\
        0 \\
        -1 \\
        1 \\
        1 \\
        0 \\
        0 \\
        1 \\
        0 \\
        -1 \\
        -1 \\
        0 \\
        1 \\
        -1 \\
        0 \\
        0
    \end{pmatrix}
$$

# Scramble matrices
In order to solve a puzzle, the pieces have to be moved around and rotated. I will refer to a series of movements and rotations as a scramble. These scrambles can be represented as matrices, which can be applied to the puzzle vector. A scramble matrix can be constructed as follows:
* Start with an $n\times n$ matrix with all entries set to 0, where n is the number of puzzle piece sides (and thus the length of vector $\vec{p}$).
* For each side in the puzzle, find its original position $a$ and its scrambled position $b$. Set the entry at row $b$ and column $a$ to 1.

There are a few constraints to this scramble matrix. Each row and each column in the matrix can only have a single entry that is equal to one, since all sides in the original puzzle have to be present in the scrambled puzzle, and sides can not be duplicated or destroyed. It is also not possible to move sides independently, since they are connected to puzzle pieces. This means that each $4\times4$ section of the scramble matrix can only take one of the following forms:

$$
    \mathbf{R}_0 = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{pmatrix},
    \mathbf{R}_1 = \begin{pmatrix}
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_2 = \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_3 = \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0
    \end{pmatrix},
    \mathbf{\phi} = \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$
If the submatrix starting at row $4i$ and column $4j$ in the scramble matrix looks like $\mathbf{R}_0$, then the scramble involves moving piece number $j$ to the position of piece number $i$ without rotating it. If the submatrix looks like $\mathbf{R}_1$ then piece $j$ was moved to the position of piece $i$ and rotated once counter-clockwise. Likewise, $\mathbf{R}_2$ means the piece has been moved and rotated twice, while $\mathbf{R}_3$ means the piece has been moved and rotated counter-clockwise three times (equivalent to rotating clockwise once). Submatrix $\mathbf{\phi}$ means that piece $j$ was not moved to the position of piece $i$.

As an example, the following scramble matrix involves rotating the first piece of a $2\times2$ puzzle once counter-clockwise, and then swapping the positions of the first and fourth pieces.
$$
    \mathbf{T} = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    \end{pmatrix}
$$

# Validating if a puzzle is solved
A puzzle can be validated by checking each side of the puzzle: if the side is on an outside edge of the puzzle, it must have a connection shape of 0, while if the side faces another side in the puzzle, the sum of the shapes of the two sides must be 0. This set of checks can be encoded as a verification matrix, such that a puzzle is solved if and only if its puzzle vector $\vec{p}$ satisfies the equation
$$
    \mathbf{V}\vec{p} = \vec{0}
$$
Each row in the verification matrix corresponds to checking a single side in the puzzle. If that side faces another side in the puzzle, the the column for each of those sides is set to one. If the side is on the outside edge of the puzzle, then only the column corresponding to that side is set to one. Each row that corresponds to checking two sides that face each other is repeated twice, since the check is made once for each side in the pair. It is useful to order the rows so that row number $i$ corresponds to checking if side number $i$ is valid, since this will make each entry on the main diagonal of the matrix have the value one, and will also make the matrix symmetric about the main diagonal.

It is notable that since $\mathbf{V}\vec{p} = \vec{0}$ has many non-trivial solutions for $\vec{p}$ (any valid puzzle is a solution), the verification matrix $\mathbf{V}$ must be non-invertible.

As an example, the verification matrix for a 2x2 puzzle looks like this:
$$
    \mathbf{V} = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$

In [1]:
# %load src/verification_matrix.py
import numpy as np


def generate_verification_matrix(width: int, height: int) -> np.array:
    number_of_sides = 4*width*height
    # The verification matrix always uses the value of the entry itself, so start with an identity matrix
    verification_matrix = np.identity(number_of_sides, np.int16)

    # Go through each puzzle piece and add its connection to the right and down
    for x in range(width):
        for y in range(height):
            piece_position = y*width + x
            # Don't add right connection if we're in the rightmost row
            if x != width - 1:
                other_piece_position = y*width + x + 1
                right_side_of_left_piece = 4*piece_position + 2
                left_side_of_right_piece = 4*other_piece_position
                verification_matrix[right_side_of_left_piece][left_side_of_right_piece] = 1
                verification_matrix[left_side_of_right_piece][right_side_of_left_piece] = 1
            # Also don't add down connection if we're in the bottom row
            if y != height - 1:
                other_piece_position = (y+1)*width + x
                bottom_side_of_top_piece = 4*piece_position + 3
                top_side_of_bottom_piece = 4*other_piece_position + 1
                verification_matrix[top_side_of_bottom_piece][bottom_side_of_top_piece] = 1
                verification_matrix[bottom_side_of_top_piece][top_side_of_bottom_piece] = 1

    return verification_matrix


 # Finding puzzles with multiple solutions
 Any arbitrary scramble (i.e. movement and/or rotation of the pieces) of a puzzle can be described using a scramble matrix, $\mathbf{T}$. If the matrix equation $\mathbf{V}\mathbf{T}\vec{p} = \vec{0}$ holds, then this scramble is a solution to the puzzle. For a given valid scramble (more on that in the next section) $\mathbf{T}$ and a given verification matrix $\mathbf{V}$, it is possible to construct a puzzle vector $\vec{p}$ such that the following system of matrix equations hold true:
 $$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
 $$
 $$
    \mathbf{V}\vec{p} = \vec{0}
 $$

A puzzle $\vec{p}$ that satisfies these constraints must have at least two solutions, since both its unscrambled state and its state that is scrambled according to $\mathbf{T}$ are solutions.

In [2]:
# %load src/puzzle_generator.py
from typing import Dict, Set
import numpy as np


# Generating a solvable puzzle involves finding the constraints given by a verification matrix and a scramble
# matrix. However, the unscrambled constraints are always the same for a given verification matrix
unscrambled_constraints = {}


def get_constrained_pairs(
        constraint_matrix: np.array,
        existing_pairs: Dict[int, Set[int]] = None
) -> Dict[int, Set[int]]:
    pairs = {}
    # Deep copy the existing pairs, if they exist
    if existing_pairs is not None:
        for side in existing_pairs:
            pairs[side] = set(existing_pairs[side])

    for row in constraint_matrix:
        constrained_sides = np.nonzero(row)[0]
        # If this side is not constrained by any other, it is an edge piece.
        # We'll ignore it and leave its value at 0
        if len(constrained_sides) != 2:
            continue

        for i in range(2):
            side = constrained_sides[i]
            other_side = constrained_sides[(i + 1) % 2]
            if side not in pairs:
                pairs[side] = set()
            pairs[side].add(other_side)
    return pairs


def generate_solvable_puzzle(verification_matrix: np.array, scramble_matrix: np.array) -> np.array:
    # Both the verification matrix and the scramble matrix are square matrices, with a number of rows equal to the
    # number of sides in the puzzle
    trial_solution = np.zeros((len(verification_matrix), 1), np.int16)

    # Find each pair of sides that are constrained by each other in the unscrambled puzzle
    pairs = unscrambled_constraints.get(id(verification_matrix), None)
    if pairs is None:
        pairs = get_constrained_pairs(verification_matrix)
        unscrambled_constraints[id(verification_matrix)] = pairs

    pairs = get_constrained_pairs(verification_matrix @ scramble_matrix, pairs)

    # Pick a side that does not have a set shape, set an arbitrary shape,
    # set all other sides that are constrained by this.
    # Repeat until no unset sides are left
    unset_sides = set(side for side in pairs)
    shape_number = 1

    while len(unset_sides) > 0:
        starting_side = unset_sides.pop()
        trial_solution[starting_side] = shape_number
        shape_number += 1

        affected_sides = [starting_side]
        affected_side_counter = 0
        while affected_side_counter < len(affected_sides):
            current_side = affected_sides[affected_side_counter]
            unset_sides.discard(current_side)
            for other_side in pairs[current_side]:
                trial_solution[other_side] = -trial_solution[current_side]
                if other_side not in affected_sides:
                    affected_sides.append(other_side)

            affected_side_counter += 1

    return trial_solution


# Random scrambles and scramble similarity
Since we now know how to generate a puzzle that has at least two solutions given an arbitrary scramble, we can generate a random scramble and build a puzzle with multiple solutions using that. However, it is important to keep in mind that the scramble can't be completely arbitrary: only corner pieces can occupy the corners of the jigsaw puzzle, and only edge pieces can occupy the edges, and their orientation is determined by which edge they're on. All middle pieces can be arbitrarily rearranged and rotated. Any scramble that conforms to these rules is valid, and can be used to construct a puzzle vector $\vec{p}$.

In the video, Matt Parker also wants the two solutions to a puzzle to be as different as possible. One way of quantifying how similar the puzzle is after being scrambled is to count how many sides are adjacent to each other in the scrambled puzzle that were also adjacent in the original puzzle. This is easily countable: if every connecting shape in the original puzzle is unique, then every pair of sides that is still able to connect after scrambling the puzzle are sides that were connected in the original puzzle. This means that if a puzzle vector $\vec{p}_u$ is constructed so that no two connections have the same shape, the number of edges in the puzzle is denoted as $n_\text{edges}$, and $n_\text{non-zero}$ is equal to the number of non-zero entries in the vector $\mathbf{V}\mathbf{T}\vec{p}_u$, the similarity of the puzzle after being scrambled is equal to
$$
    s = \frac{n_\text{non-zero}-n_\text{edges}}{2}
$$

In [3]:
# %load src/scrambles.py
from typing import Tuple
import numpy as np
import random


def categorize_position(width: int, height: int, x: int, y: int) -> Tuple[str, int]:
    if y == 0:
        if x == 0:
            return 'corner', 0
        elif x == width - 1:
            return 'corner', 3
        else:
            return 'edge', 0
    elif y == height - 1:
        if x == 0:
            return 'corner', 1
        elif x == width - 1:
            return 'corner', 2
        else:
            return 'edge', 2
    elif x == 0:
        return 'edge', 1
    elif x == width - 1:
        return 'edge', 3
    else:
        return 'middle', 0


# Store already calculated rotation matrices, to save on computation
rotation_matrices = {}


def generate_random_scramble(width: int, height: int) -> np.array:
    number_of_sides = 4 * width * height
    scramble = np.zeros((number_of_sides, number_of_sides), np.int16)

    # Group pieces in corners, edges and middle, shuffle them, and then fill them in using the shuffled order
    corners = []
    edges = []
    middles = []
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            category, _ = categorize_position(width, height, x, y)
            if category == 'corner':
                corners.append(piece_number)
            elif category == 'edge':
                edges.append(piece_number)
            elif category == 'middle':
                middles.append(piece_number)

    random.shuffle(corners)
    random.shuffle(edges)
    random.shuffle(middles)

    # Now fill in the puzzle
    for scrambled_x in range(width):
        for scrambled_y in range(height):
            category, orientation = categorize_position(width, height, scrambled_x, scrambled_y)
            if category == 'corner':
                piece_number = corners.pop()
            elif category == 'edge':
                piece_number = edges.pop()
            else:
                piece_number = middles.pop()

            original_x = piece_number % width
            original_y = piece_number // width
            _, original_orientation = categorize_position(width, height, original_x, original_y)
            rotation = (orientation - original_orientation) % 4

            # If the piece is a middle piece, we can rotate it freely
            if category == 'middle':
                rotation = random.randrange(4)

            rotation_matrix = rotation_matrices.get(rotation, None)
            if rotation_matrix is None:
                # Generate a rotation matrix, and place it in the correct position in the larger scramble matrix
                # Each rotation is counter-clockwise and 90 degrees. We can achieve this by shifting each row
                # in the identity matrix one up
                rotation_matrix = np.roll(np.identity(4, np.int16), -rotation, 0)
                rotation_matrices[rotation] = rotation_matrix

            # Place the rotation matrix in the correct spot in the larger scramble matrix
            new_piece_number = scrambled_y * width + scrambled_x
            scramble[
                4 * new_piece_number:4 * new_piece_number + 4,
                4 * piece_number:4 * piece_number + 4
            ] = rotation_matrix

    return scramble


# Store pre-generated unique puzzles. This saves quite a lot of execution time
generated_puzzles = {}


def get_unique_puzzle(verification_matrix: np.array) -> np.array:
    unique_puzzle, number_of_edges = generated_puzzles.get(id(verification_matrix), (None, None))

    if unique_puzzle is None:
        # Generate a puzzle where each connection is unique
        unique_puzzle = np.zeros((len(verification_matrix), 1), np.int16)
        number_of_edges = 0
        shape_number = 1
        for row in verification_matrix:
            connected_sides = []
            for index, value in enumerate(row):
                if value != 0:
                    connected_sides.append(index)
            if len(connected_sides) == 1:
                number_of_edges += 1
            else:
                unique_puzzle[connected_sides[0]] = shape_number
                unique_puzzle[connected_sides[1]] = -shape_number
                shape_number += 1
        generated_puzzles[id(verification_matrix)] = (unique_puzzle, number_of_edges)
    return unique_puzzle, number_of_edges


def find_scramble_similarity(verification_matrix: np.array, scramble_matrix: np.array) -> int:
    # The unique_puzzle is a puzzle where the shapes of every connection is unique
    unique_puzzle, number_of_edges = get_unique_puzzle(verification_matrix)
    # Now, the number of sides that are still touching in the scrambled puzzle is equal to the number of zeros in the
    # vector generated by the verification matrix, minus the number of edges (which are always zero in the vector)
    verification_vector = verification_matrix @ scramble_matrix @ unique_puzzle
    similarity = -number_of_edges
    for entry in verification_vector:
        if entry == 0:
            similarity += 1
    # Each similarity is counted twice in the verification vector
    return similarity // 2


# Counting the number of solutions
In order to achieve the effect in the video where the puzzle motif changes from a coffee cup to a donut, the puzzle should only have two solutions, otherwise it would be possible to put it back together in different ways that create nonsensical motifs. Since the above method guarantees that a puzzle has *at least* two solutions, we need to count the number of possible solutions to ensure that the puzzle has *only* two solutions.

Counting the number of possible solutions amounts to finding the number of matrices $\mathbf{T}$ that satisfies the equation
$$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
$$
where $\mathbf{V}$ and $\vec{p}$ are known. I don't think there exists a better way of finding the number of solutions than trial and error. My explanation for this is given in the next cell in this notebook, which I have hidden by default.

Since counting the number of solutions takes a long time, it is useful to weed out puzzles that obviously have more than two solutions. Any where two pieces have the exact same side shapes must have at least two solutions, since we can solve the puzzle by simply swapping those two pieces. Since we already know that our chosen puzzle also has a solution that is more complicated than just swapping two pieces, our puzzle must have more than two solutions.

It also seems likely that the more often a particular connection shape is repeated, the more likely a puzzle is to have many solutions. Additionally, puzzles with fewer repeated connections should be faster to check, and Matt Parker also states in the video that the most satisfying puzzle would be one with only two solutions, no shared sides, and no more than two of any connection type. This seems almost impossible to find, but setting an upper bound on the number of repeated connection shapes will also weed out a lot of puzzles.

# Attempt at finding $\mathbf{T}$ through linear algebra
In order to express our constraints on the entries of the scramble matrix $\mathbf{T}$, we can unroll its entries, row by row, into a scramble vector $\vec{t}$ so that the entries of $\vec{t}$ look like:
$$
    \vec{t} = \begin{pmatrix}
        T_{1,1} \\
        T_{1,2} \\
        \vdots \\
        T_{1,n} \\
        T_{2,1} \\
        \vdots \\
        T_{3,1} \\
        \vdots \\
        T_{n,n}
    \end{pmatrix}
$$
We can encode our constraints into a matrix $\mathbf{C}$, such that $\mathbf{T}$ is only a valid scramble matrix if its entry vector $\vec{t}$ satisfies the following equation:
$$
    \mathbf{C}\vec{t}=\vec{b}
$$
I'll illustrate what constraints can be encoded into $\mathbf{C}$ and $\vec{b}$ in the next section.

We already know that $\mathbf{C}\vec{t}=\vec{b}$ must be an underdetermined system, since there exists several scrambles that solves the puzzle. This means that $\mathbf{C}$ has no inverse. However, $\mathbf{C}$ must have a unique pseudo-inverse $\mathbf{C}^\dagger$, and using this we can determine that every solution of $\vec{t}$ must lie in the space 
$$
    \vec{v} = \mathbf{C}^\dagger\vec{b} + \left[\mathbb{I} - \mathbf{C}^\dagger\mathbf{C}\right]\vec{\omega}
$$
where $\vec{\omega}$ is an arbitrary vector (see [the article on the Moore-Penrose inverse on Wikipedia](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Applications)). However, even though $\vec{v}$ has infinitely many solutions, we are only looking for the ones where all the values of $\vec{t}$ are either $0$ or $1$. I am really not an expert on multidimensional geometry, but I suspect that is is possible to define some other subspaces where the entries of $\vec{t}$ are locked to values that correspond to putting certain pieces down, and then checking whether this space of possible scramble vectors $\vec{t}$ and the solution space intersect. I also suspect that this is a very difficult and computationally expensive operation, but, again, I am really not an expert on multidimensional geometry. Worst case, one could find the points in the vector space of $\vec{t}$ that corresponds to every single permutation of the puzzle, and check which of those points lie inside the solution space, but this is worse than trial and error.

Another problem with the linear algebra method of solving for the scramble matrix is the size of the data structures involved. Since the scramble matrix has dimensions $4n\times 4n$, where $n$ is the number of pieces in the puzzle, the scramble vector has dimensions $16n^2\times1$. This again means that the matrix $\mathbf{C}^\dagger\mathbf{C}$ will have dimensions of $16n^2\times16n^2$, which is getting prohibitively large.

## An estimate on how many pieces it is necessary to place before the scramble matrix can be determined
As mentioned previously, it is possible to encode constraints on the scramble matrix so that every valid scramble matrix must satisfy the equation
$$
    \mathbf{C}\vec{t}=\vec{b}
$$
As also mentioned previously, as long as this system of equations is underdetermined, there also exists an infinity of an invalid scramble matrices that satisfy the equation. In order for the system of equations above to have a single solution, the number of linearly independent equations encoded in the constraint matrix $\mathbf{C}$ must be equal to or larger than the number of entries in the scramble vector $\vec{t}$. Some constraints follow from the fact that we want the scramble matrix to be valid and the puzzle to be solved, while additional constraints can be added to the system by locking puzzle pieces into place in the scramble.

#### Constraints due to connection shapes
Before any pieces are placed, there are already some constraints on the scramble matrix. Any valid scramble matrix must move the pieces so that only sides with compatible shapes end up next to each other. For example, if entry number 2 and 4 are adjacent in the puzzle vector, rows 2 and 4 in the scramble matrix are both constrained by the equation
$$
    (T_{2,1} + T_{4,1})p_1 + (T_{2,2} + T_{4,2})p_2 + \dots + (T_{2,n} + T_{4,n})p_n = 0
$$
This can be encoded in the constraint matrix as
$$
    \begin{pmatrix}
        \dots & 0 & p_1 \dots & p_n & 0 & \dots & 0 & p_1 & \dots & p_n & 0 & \dots
    \end{pmatrix}
    \begin{pmatrix}
        \vdots \\ T_{1,n} \\ T_{2,1} \\ \vdots \\ T_{2,n} \\ T_{3,1} \\ \vdots \\ T_{3,n} \\ T_{4,1} \\ \vdots T_{4,n} \\ T_{5,1} \\ \vdots
    \end{pmatrix}
    = \begin{pmatrix}
        0
    \end{pmatrix}
$$
Since constraints need to be linearly independent in order to contribute to a solution, it is not useful to add more than one constraint for each pair of sides facing each other. The constraints on the outside edges of the puzzle are also not linearly independent of these constraints: all the non-zero entries in the puzzle vector must be used to satisfy these constraints, meaning that the outside edges must automatically all have the value 0.

If the width of the puzzle is $w$ and the height is $h$, there are $2wh - w - h$ inside connections in the puzzle, meaning we can construct $2wh - w -h$ rows in the constraint matrix.

#### Constraints on the rows and columns of the scramble matrix
The scramble matrix cannot have arbitrary entries. As mentioned previously each row and each column can only have a single entry with the value of one. This can be encoded in the constraint matrix as restricting the sum of all entries in each row and in each column of the scramble matrix to be one. How to do this is left as an exercise to the reader.

This gives an additional $2\times4wh$ constraints on the puzzle, where $4wh$ is equal to the number of rows (and columns) in the scramble matrix. However, these constraints are not all linearly independent. If we know all but one of these constraints, it is possible to deduce the final constraint. Therefore, the number of constraints we can add to the constraint matrix is $8wh - 1$.

#### Constraints on piece movement
As mentioned previosly, each $4\times4$ section of the scramble matrix can only take the form $\mathbf{R}_0$, $\mathbf{R}_1$, $\mathbf{R}_2$, $\mathbf{R}_3$ or $\mathbf{\phi}$. Given the following section of the transformation matrix
$$
    \begin{pmatrix}
        T_{1,1} & T_{1,2} & T_{1,3} & T_{1,4} \\
        T_{2,1} & T_{2,2} & T_{2,3} & T_{2,4} \\
        T_{3,1} & T_{3,2} & T_{3,3} & T_{3,4} \\
        T_{4,1} & T_{4,2} & T_{4,3} & T_{4,4}
    \end{pmatrix}
$$
this constraint can be added by requiring that
$$
    \begin{matrix}
        T_{1,1} - T_{2,2} = 0, & T_{2,2} - T_{3,3} = 0, & T_{3,3} - T_{4,4} = 0, & T_{4,4} - T_{1,1} = 0 \\
        T_{2,1} - T_{3,2} = 0, & T_{3,2} - T_{4,3} = 0, & T_{4,3} - T_{1,4} = 0, & T_{1,4} - T_{2,1} = 0 \\
        T_{3,1} - T_{4,2} = 0, & T_{4,2} - T_{1,3} = 0, & T_{1,3} - T_{2,4} = 0, & T_{2,4} - T_{3,1} = 0 \\
        T_{4,1} - T_{1,2} = 0, & T_{1,2} - T_{2,3} = 0, & T_{2,3} - T_{3,4} = 0, & T_{3,4} - T_{4,1} = 0
    \end{matrix}
$$
However, these equations are not fully linearly independent, as can be seen by
$$
    \left(T_{1,1} - T_{2,2}\right) + \left(T_{2,2} - T_{3,3}\right) + \left(T_{3,3} - T_{4,4}\right) = T_{1,1} - T_{4,4} = 0
$$
and similarly for the three other rows in the above system of equations. This means that for every four entries in the scramble matrix, there are three linearly independent constraints. This gives a total number of constraints of this type at $12w^2h^2$.

### Sum of constraints
In total, before any pieces are placed, the total number of constraints $n_c$ is therefore
$$
    \begin{align}
        n_c &= (2wh - w - h) + (8wh - 1) + 12w^2h^2 \\
        n_c &= 12w^2h^2 + 10wh - w - h - 1
    \end{align}
$$
There are $16w^2h^2$ entries in the scramble matrix, meaning that the system has fewer constraints than unknowns for every puzzle larger than $2\times1$.

### Locking in puzzle pieces
Locking in a puzzle piece in the scramble matrix means that every other entry in the rows and columns affected in the scramble matrix also have their values locked. This means that locking a piece in place locks the value $2\times4\times4wh - 16 = 32wh - 16$ entries in the scramble matrix. (The $-16$ term is to avoid double counting the constraints on the entry at the intersection of the row and column that is locked). However, the previous constraints on the entries in those rows and columns are now redundant, as are the constraints on the rows and columns themselves. This means that $\frac{3}{4}(2\times4\times4wh-16) = 24wh - 12$ constraints on all the entries in the rows and columns can be removed, and $8$ constraints on the sums of the rows and columns affected. If the piece is placed next to one more neigbouring pieces, there may also be some constraints due to connection shapes that are now redundant, but we'll ignore that for now in order to obtain a maximum bound on the number of constraints. This means that the sum of added constraints on matrix entries when placing the first piece is $8wh - 4$, while the sum of removed constraints on the rows and columns of the matrix is $8$.

However, care must be taken when placing subsequent puzzle pieces, as each subsequent locked row and column will intersect with each of the previously locked rows and columns. This means that for each subsequent locked piece, we must avoid double counting the matrix entries that were already locked when placing a previous piece. This requires a function of the number of placed pieces $n_p$ that takes the form
$$
    f(n_p) = f(n_p-1) + (8wh - 4) - 2\times4(n-1)
$$
where the term $f(n_p-1)$ is the number of entries that were already locked in by placing the previous puzzle piece, $(4wh+4)$ are the additional locked entries from placing piece number $n$ and $2\times4(n-1)$ is the number of entries that must not be double counted due to intersections with previously locked rows and columns. The following function satisifies the above equation:
$$
    f(n_p) = 8whn - 4n^2
$$

This gives an upper bound for the number of constraints after adding n pieces as
$$
    n_c(n_p) = 12w^2h^2 + 10wh - w - h - 1 + 8whn - 4n^2 - 8n
$$
(where the final $8n$ term is to account for the row and column restrictions that are made redundant by each piece placed).

In order for the scramble matrix to be determined, the number of constraints must be larger than the number of entries in the matrix:
$$
    \begin{align}
        16w^2h^2 &< 12w^2h^2 + 10wh - w - h - 1 + 8whn_p - 4n_p^2 - 8n_p \\
        0 &< -4n_p^2 + (8wh - 8)n_p -4w^2h^2 + 10wh - w - h - 1
    \end{align}
$$
This is a quadratic equation, which can be easily solved. The following table shows the number of pieces that must be fixed before the scramble matrix only has a single solution for various sizes of puzzle:

| Size of puzzle | Necessary number of pieces before system is overdetermined | Total pieces |
| -------------- | ---------------------------------------------------------- | ------------ |
| $5\times5$     | 20.72                                                      | 25           |
| $5\times6$     | 25.39                                                      | 30           |
| $6\times6$     | 31.03                                                      | 36           |
| $6\times7$     | 36.70                                                      | 42           |
| $7\times7$     | 43.34                                                      | 49           |
| $10\times10$   | 92.24                                                      | 100          |
| $100\times100$ | 9928.64                                                    | 10 000       |

As can be seen from the table, you need to have so many pieces already in placed before the puzzle unambigously solved that I don't think there is much to gain from trying to solve it using linear algebra. Unless, of course, there are more constraints that I have forgotten to include, which might bring the number of necessary pieces down.

In [4]:
# %load src/solution_finder.py
from typing import List, Dict, Tuple
import numpy as np


def get_max_repeated_shapes(puzzle: np.array) -> int:
    shape_type_count = [[int(s) for s in puzzle[:, 0]].count(i) for i in range(1, max(puzzle[:, 0]))]
    return max(shape_type_count)


def has_duplicate_pieces(puzzle: np.array) -> bool:
    pieces = []
    for i in range(0, len(puzzle), 4):
        piece = (puzzle[i], puzzle[i + 1], puzzle[i + 2], puzzle[i + 3])
        rotations = [(puzzle[i + j % 4], puzzle[i + (j + 1) % 4], puzzle[i + (j + 2) % 4], puzzle[i + (j + 3) % 4]) for
                     j in range(4)]
        for other_piece in pieces:
            if other_piece in rotations:
                return True
        pieces.append(piece)
    return False


class PuzzlePiece:
    def __init__(self, piece_number: int, connection_shapes: List[int]):
        self.piece_number = piece_number
        self.connection_shapes = connection_shapes

    def get_side_shape(self, side_number: int, rotation_number: int):
        return self.connection_shapes[(side_number - rotation_number) % 4]


class PuzzleSolutionBuilder:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.placed_pieces = {}
        self.unfilled_neighbours = {}

    def place_piece(self, piece: PuzzlePiece, rotation: int, position_x: int, position_y: int) -> bool:
        neighbour_restrictions = []
        position = position_y * self.width + position_x

        # Check with neighbours that it fits
        for dx, dy, our_side_facing_neighbour, neighbour_side_facing_us in [(1, 0, 2, 0), (0, 1, 3, 1), (-1, 0, 0, 2),
                                                                            (0, -1, 1, 3)]:
            neighbour_x = position_x + dx
            neighbour_y = position_y + dy
            # Check we're still inside the board
            if (not 0 <= neighbour_x < self.width) or (not 0 <= neighbour_y < self.height):
                # If we're not, our side facing this way must have a shape of zero
                if piece.get_side_shape(our_side_facing_neighbour, rotation) != 0:
                    return False
                continue

            # Check if a neighbouring piece has been placed yet
            neighbour_position = neighbour_y * self.width + neighbour_x
            if neighbour_position in self.placed_pieces:
                # If there is a piece in the neighbouring location, check that this piece is compatible with that
                neighbour_piece, neighbour_rotation = self.placed_pieces[neighbour_position]
                if (
                        neighbour_piece.get_side_shape(neighbour_side_facing_us, neighbour_rotation)
                        + piece.get_side_shape(our_side_facing_neighbour, rotation)
                        != 0
                ):
                    # If we get here, the piece did not fit
                    return False
            else:
                # If there is no neighbour on this side, we should store what shape that neighbour needs to fit
                neighbour_restrictions.append((
                    neighbour_position,
                    neighbour_side_facing_us,
                    -piece.get_side_shape(our_side_facing_neighbour, rotation)
                ))

        # If we went through all the sides, the piece fits. Place it
        self.placed_pieces[position] = (piece, rotation)

        # This is no longer an unfilled neighbour
        if position in self.unfilled_neighbours:
            del self.unfilled_neighbours[position]

        # Add any new unfilled neighbours we found
        for neighbour_position, neighbour_side_facing_us, required_shape in neighbour_restrictions:
            if neighbour_position not in self.unfilled_neighbours:
                self.unfilled_neighbours[neighbour_position] = []
            self.unfilled_neighbours[neighbour_position].append((neighbour_side_facing_us, required_shape))

        return True

    def remove_last_piece(self) -> PuzzlePiece:
        piece_position, (piece, rotation) = self.placed_pieces.popitem()

        requirements_for_replacement_piece = []

        piece_x = piece_position % self.width
        piece_y = piece_position // self.width
        # Update the unfilled neighbours
        for dx, dy, our_side_facing_neighbour, neighbour_side_facing_us in [(1, 0, 2, 0), (0, 1, 3, 1), (-1, 0, 0, 2),
                                                                            (0, -1, 1, 3)]:
            neighbour_x = piece_x + dx
            neighbour_y = piece_y + dy

            # If this side is off the board, we will not need to do anything about any requirements
            if (not 0 <= neighbour_x < self.width) or (not 0 <= neighbour_y < self.height):
                continue

            # If there is still a neighbouring piece on this side, we need to require whichever piece
            # replaces this piece to fit with that neighbour
            neighbour_position = neighbour_y * self.width + neighbour_x
            if neighbour_position in self.placed_pieces:
                requirements_for_replacement_piece.append((
                    our_side_facing_neighbour,
                    piece.get_side_shape(our_side_facing_neighbour, rotation)
                ))
                continue

            # If there is no neighbour on this side, there should be an unfilled neighbour with a set of requirements
            # there. Remove our requirements for that neighbour, and if there are no requirements left, remove the
            # unfilled neighbour completely
            our_requirement = (neighbour_side_facing_us, -piece.get_side_shape(our_side_facing_neighbour, rotation))
            self.unfilled_neighbours[neighbour_position].remove(our_requirement)
            if len(self.unfilled_neighbours[neighbour_position]) == 0:
                del self.unfilled_neighbours[neighbour_position]

        # Leave the requirements for any replacement pieces
        self.unfilled_neighbours[piece_position] = requirements_for_replacement_piece
        return piece

    def get_unfilled_neighbours(self) -> Dict[int, Tuple[int, int]]:
        return self.unfilled_neighbours

    def get_number_of_pieces(self) -> int:
        return len(self.placed_pieces)

    def is_finished(self) -> bool:
        return len(self.placed_pieces) == self.width * self.height


def count_solutions(
        puzzle: np.array,
        width: int,
        height: int
) -> int:
    # Do this in the same way Matt Parker does in the video. Just try every combination of pieces until we find one that
    # works. Keep a running tally of solutions until we are done
    number_of_solutions = 0

    # Create pieces based on the puzzle input
    pieces = []
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            side_shapes = [int(shape) for shape in puzzle[4 * piece_number:4 * piece_number + 4, 0]]
            pieces.append(PuzzlePiece(piece_number, side_shapes))

    # Create a map from a given shape to the pieces that have that shape
    shape_to_pieces_map = {}
    for piece in pieces:
        for side_number in range(4):
            shape = piece.get_side_shape(side_number, 0)
            if shape not in shape_to_pieces_map:
                shape_to_pieces_map[shape] = set()
            shape_to_pieces_map[shape].add((piece, side_number))

    solution_builder = PuzzleSolutionBuilder(width, height)

    # We want to do a depth first search of the puzzle
    # Store it every time we have to make a choice, so we can go back later and do the opposite choice
    available_pieces = set(pieces)
    # Each choice stores which piece to place, its x and y position, rotation, and how many pieces were already
    # laid when making the choice
    choices = [(pieces[0], 0, 0, 0, 0)]
    while len(choices) > 0:
        piece_to_place, position_x, position_y, rotation, pieces_placed_when_making_choice = choices.pop()
        # Since we're doing a depth first search, we only need to remove pieces to get back to the state we were
        # in when we made this choice
        while solution_builder.get_number_of_pieces() > pieces_placed_when_making_choice:
            removed_piece = solution_builder.remove_last_piece()
            # The piece we just removed is available to be placed again
            available_pieces.add(removed_piece)

        # Try to place the piece
        if not solution_builder.place_piece(piece_to_place, rotation, position_x, position_y):
            # If we couldn't place the piece, this is a dead end, and we must return
            continue

        # The piece we placed is no longer available
        available_pieces.remove(piece_to_place)

        # We might have finished building the puzzle now
        if solution_builder.is_finished():
            number_of_solutions += 1
            continue

        # Check what options we have for placing the next puzzle piece
        options = {}
        for neighbour_position in solution_builder.get_unfilled_neighbours():
            requirements = solution_builder.get_unfilled_neighbours()[neighbour_position]
            # To start with, I'll just find all the pieces that fit one of the requirements
            # Maybe we can do more requirement checking later
            required_side, required_shape = requirements[0]

            # Find all pieces with the required shape, and make sure we only consider pieces that are available
            possible_pieces = shape_to_pieces_map[required_shape]

            # Find the possible rotations for each possible piece
            options[neighbour_position] = []
            for possible_piece, side_number in possible_pieces:
                if possible_piece in available_pieces:
                    options[neighbour_position].append((possible_piece, required_side - side_number))

        # Place a piece in the position with the fewest options
        best_position = min(options, key=lambda position: len(options[position]))
        best_position_x = best_position % width
        best_position_y = best_position // width
        # Create a choice for each possible piece
        for piece, rotation in options[best_position]:
            choices.append((piece, best_position_x, best_position_y, rotation, solution_builder.get_number_of_pieces()))
    return number_of_solutions


The following cell contains code to draw the puzzle

In [5]:
# %load src/puzzle_drawer.py
from ipycanvas import Canvas
from PIL import Image
import numpy as np


def draw_puzzle(puzzle, width, height, scramble=None):
    if scramble is None:
        canvas = Canvas(width=100 * width, height=100 * height, sync_image_data=True)
    else:
        canvas = Canvas(width=200 * width + 50, height=100 * height, sync_image_data=True)

    image = Image.open('img/matt_and_steve.png')

    # Crop depending on puzzle aspect ratio
    desired_width_pixels = 100 * width
    desired_height_pixels = 100 * height

    width_resize = desired_width_pixels / image.width
    height_resize = desired_height_pixels / image.height

    if height_resize > width_resize:
        image = image.resize((int(height_resize * image.width), int(height_resize * image.height)), Image.LANCZOS)

        left_crop = 110 * height_resize
        if image.width - left_crop < 100 * width:
            left_crop = image.width - 100 * width
        image = image.crop((int(left_crop), 0, int(left_crop) + 100 * width, 100 * height))
    else:
        image = image.resize((int(width_resize * image.width), int(width_resize * image.height)), Image.LANCZOS)

        top_crop = 100 * width_resize
        if image.height - top_crop < 100 * height:
            top_crop = image.height - 100 * height
        image = image.crop((0, int(top_crop), 100 * width, int(top_crop) + 100 * height))

    # Cut image into pieces
    image_pieces = {}
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            image_piece = image.crop((x * 100, y * 100, (x + 1) * 100, (y + 1) * 100))
            image_pieces[piece_number] = image_piece

    # Draw original puzzle
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            image_piece = image_pieces[piece_number]
            canvas.put_image_data(np.asarray(image_piece), x * 100, y * 100)

    # Draw borders between puzzle pieces
    canvas.stroke_style = 'black'
    for x in range(width + 1):
        canvas.stroke_line(x * 100, 0, x * 100, 100 * height)
    for y in range(height + 1):
        canvas.stroke_line(0, y * 100, width * 100, y * 100)

    # Draw piece number at the centre of each piece
    canvas.stroke_style = 'white'
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            center_x = 100 * x + 50
            center_y = 100 * y + 50
            canvas.stroke_text(str(piece_number), center_x, center_y)

    # Draw shape type at the edge of each piece
    canvas.stroke_style = 'red'
    directions = {
        0: (-42, 0),
        1: (0, -40),
        2: (42, 0),
        3: (0, 40)
    }

    for index, value in enumerate(puzzle[:, 0]):
        if value == 0:
            continue
        piece_number = index // 4
        center_x = 100 * (piece_number % width) + 50
        center_y = 100 * (piece_number // width) + 50

        direction = index % 4
        dx, dy = directions[direction]

        canvas.stroke_text(str(value), center_x + dx, center_y + dy)

    # Draw a dot above each number, to show that pieces are in their original orientation
    canvas.fill_style = 'green'
    for x in range(width):
        for y in range(height):
            location_x = x * 100 + 50
            location_y = y * 100 + 35
            canvas.fill_circle(location_x, location_y, 3)

    # If there is no scramble, we are done now
    if scramble is None:
        return canvas

    # Else, map where each piece has gone to and whether it is rotated
    scramble_map = {}
    for new_piece_number in range(width * height):
        column_set_to_one = min(
            index for index, value in enumerate(scramble[new_piece_number * 4, :]) if value != 0)
        old_piece_number = column_set_to_one // 4
        piece_rotation = column_set_to_one % 4
        scramble_map[new_piece_number] = (old_piece_number, piece_rotation)

    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            old_piece_number, rotation = scramble_map[piece_number]
            image_piece = image_pieces[old_piece_number]
            image_piece = image_piece.rotate(90 * rotation)
            canvas.put_image_data(np.asarray(image_piece), (width + x) * 100 + 50, y * 100)

    # Draw borders between puzzle pieces
    canvas.stroke_style = 'black'
    for x in range(width + 1):
        canvas.stroke_line((x + width) * 100 + 50, 0, (x + width) * 100 + 50, 100 * height)
    for y in range(height + 1):
        canvas.stroke_line(100 * width + 50, y * 100, 200 * width + 50, y * 100)

    # Draw piece numbers of the moved pieces
    canvas.stroke_style = 'white'
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            old_piece_number, _ = scramble_map[piece_number]

            center_x = 100 * (width + x) + 100
            center_y = 100 * y + 50

            canvas.stroke_text(str(old_piece_number), center_x, center_y)

    # Draw connection shape types on the moved pieces
    canvas.stroke_style = 'red'
    directions = {
        0: (-42, 0),
        1: (0, -40),
        2: (42, 0),
        3: (0, 40)
    }

    for index, value in enumerate((scramble @ puzzle)[:, 0]):
        if value == 0:
            continue
        piece_number = index // 4
        center_x = 100 * (width + piece_number % width) + 100
        center_y = 100 * (piece_number // width) + 50

        direction = index % 4
        dx, dy = directions[direction]

        canvas.stroke_text(str(value), center_x + dx, center_y + dy)

    # Draw green dots to show how each piece has been rotated
    canvas.fill_style = 'green'
    rotations = {
        0: (0, -15),
        1: (-15, 0),
        2: (0, 15),
        3: (15, 0)
    }
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            _, rotation = scramble_map[piece_number]

            center_x = 100 * (width + x) + 100
            center_y = 100 * y + 50

            dx, dy = rotations[rotation]
            canvas.fill_circle(center_x + dx, center_y + dy, 3)

    return canvas


# Finding puzzles
Here, we find the verification matrix for a given width and height. Then we generate random scrambles until one is found where no two sides that were adjacent in the original puzzle are adjacent after the scramble. We then find a puzzle that is solved in both its unscrambled state and the state that results from the scramble we found. We then check that this puzzle doesn't have any connections that are repeated more than the maximum. If it does not, we check that it does not have any pieces that are duplicates of each other (i.e. have the same connections in the same order). If the puzzle meets all these conditions, we try to find all solutions for it. If the puzzle has only two solutions, it is returned, and a figure is drawn.

This is doable for $5\times5$ puzzles, seems to work for $5\times6$ puzzles, and seems to get completely bogged down for $6\times6$ puzzles. I think the solution finder needs to be made much more efficient in order to handle those.

I also suspect that the best way to generate puzzles more quickly is to find some smarter way to generate the scrambles, rather than generating them randomly and checking if they meet our criteria.

Rerun all the code in this notebook to see an illustration of a puzzle with only two solutions. With the default settings, this should happen almost immediately. Lowering the value of maximum_repeated_shapes should increase the runtime, but give more satisfying puzzles with fewer connection repetitions. Increasing the value of width and height should increase the runtime drastically. I've never seen it complete for a $6\times6$ puzzle so far.

In [6]:
# %load -r 8: src/main.py
import datetime


width = 5
height = 5
maximum_repeated_shapes = 10

V = generate_verification_matrix(width, height)


def print_progress(elapsed, puzzle_count, duplicate_checked, solution_checked):
    print(f'\r{elapsed} - Tested {puzzle_count} puzzles. ', end='')
    print(f'{duplicate_checked} had {maximum_repeated_shapes} or fewer repetitions of each connection shape, ', end='')
    print(f'{solution_checked} of those had no duplicate pieces.', end='')


# I'm putting the code inside a main function since this makes it easier to profile
def main(max_time=-1):
    puzzle_count = 0
    duplicate_checked = 0
    solution_checked = 0

    search_start = datetime.datetime.now()
    seconds_elapsed = 0

    puzzle_found = None
    scramble_found = None

    while True:
        puzzle_count += 1
        current_time = datetime.datetime.now()
        elapsed = current_time - search_start
        elapsed = elapsed - datetime.timedelta(microseconds=elapsed.microseconds)
        if max_time != -1 and elapsed.total_seconds() > max_time:
            break

        if elapsed.total_seconds() > seconds_elapsed:
            seconds_elapsed = elapsed.total_seconds()
            print_progress(elapsed, puzzle_count, duplicate_checked, solution_checked)

        scramble = None
        scramble_similarity = 1
        while scramble_similarity > 0:
            scramble = generate_random_scramble(width, height)
            scramble_similarity = find_scramble_similarity(V, scramble)
        p = generate_solvable_puzzle(V, scramble)
        # Check how many of each connection type there is

        if get_max_repeated_shapes(p) > maximum_repeated_shapes:
            continue

        duplicate_checked += 1

        if has_duplicate_pieces(p):
            continue

        solution_checked += 1

        number = count_solutions(p, width, height)
        if number == 2:
            puzzle_found = p
            scramble_found = scramble
            break

    print_progress(elapsed, puzzle_count, duplicate_checked, solution_checked)
    print()
    if puzzle_found is not None:
        print('Found solution with the following puzzle vector: ')
        print(puzzle_found[:,0])
        display(draw_puzzle(puzzle_found, width, height, scramble_found))
    else:
        print('Timed out without finding solution')


main()

0:00:06 - Tested 736 puzzles. 85 had 10 or fewer repetitions of each connection shape, 4 of those had no duplicate pieces.
Found solution with the following puzzle vector: 
[ 0  0  1  2 -1  0  3  4 -3  0  3  5 -3  0  6  7 -6  0  0  1  0 -2  4 -6
 -4 -4  8  4 -8 -5 -7 -4  7 -7  5  5 -5 -1  0  3  0  6  4 -3 -4 -4  5  7
 -5  4  5  9 -5 -5  7  4 -7 -3  0  1  0  3 -8 -1  8 -7 -4 -7  4 -9 -9  5
  9 -4  4 -5 -4 -1  0  1  0  1 -3  0  3  7 -1  0  1 -5  2  0 -2  5 -6  0
  6 -1  0  0]


Canvas(sync_image_data=True, width=1050)